In [2]:
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import glob
import numpy as np
import os
import random
import imgaug as ia
from imgaug import augmenters as iaa
import cv2
from zhon import hanzi
import glob

## 预备知识
* 文件读取
* PIL使用
* os操作

## 数字游戏
`round(123 / 11, 3)`#小数点后取三位, 默认取整数  
`123 // 22 `#地板除  
`123 / 22 `#float除法  
`min(12, 34, 43)`#取最小的  
`max(34, 253, 45)`#取最大的  
`np.random.uniform(0,10)`统一分布

### 文件读取
`f.read(-1)`读取指定大小的字符串  
`f.readline()`读取一行内容  
`f.readlines()`读取所有行的内容，返回一个包含所有行的list

### os.path
`os.path.splitext("path")`分离base和ext

### glob
glob模块支持的通配符：
`glob.glob("pattern")`里面路径就可以带正则匹配符  
"xxxx/TL*.bmp"里面的模式  
**通配符	功能**     
\*	匹配0或多个字符  
**	匹配所有文件、目录、子目录和子目录里的文件（3.5版本新增）  
?	匹配1个字符，与正则表达式里的?不同   
[exp]	匹配指定范围内的字符，如：[1-9]匹配1至9范围内的字符  
[!exp]	匹配不在指定范围内的字符  


In [7]:


# a = np.array([[0,0,1,1,0,0],[0,1,1,1,1,0],[0,0,0,1,0,0],[0,0,0,0,0,0]])
# # print(a)
# sample = Image.open("手写字体数据集//hand_sample.bmp").convert("L")#bmp使用灰度读取
# print(sample.size)
# # sample.show()
# sample_arr = np.array(sample)
# sample_arr = 255 -sample_arr
# bbox = crop_img(sample_arr)
# print(bbox.shape)
# newCanvas = Image.new("RGB", size=(2500, 2500), color='white')
# Image.fromarray(bbox).show()
# draw = ImageDraw.Draw(sample)
# # draw.line(100, 200)
# # draw.rectangle(bbox, outline=2)

# sample.show()
# newCanvas.show()
# print(sample_arr)
# bbox = extract_outbox(sample_arr)
# # sample_arr[bbox[0]:bbox[2], bbox[1]:bbox[3]].astype(np.int32)
# # Image.fromarray(sample_arr).show()

(1671, 108)
(103, 1486)


注意list和array不是一个东西  
list中可能有很多个对象,但是array就只是一个对象,可能二者里面包含的数值都是一样的,但是用法上还是不能混同

>注意,如果Image对象是灰度图时,传入的所有颜色color参数只能有一个通道,如果传入元组会报错!!!!  
str 一旦初始化之后，不允许重新赋值!!assignment banned!!

In [846]:
np.maximum(4, [1,2,3,4,5])#两个array进行比较，每个都取大的元素，返回一个相同长度的array

array([4, 4, 4, 4, 5])

`np.any(axis=)`axis=0沿着另一个轴


### Image
该基类是`Image.Image`,如果需要help在这个类的基础上查看  
**方法和属性：**
`Image.size`返回(w, h)  注意与shape返回的不一样
`Image.split()=>多个通道的list`通道分离,  
`Image.resize((size))`图像尺寸变化  
`Image.convert("RGB")`图像色彩模式转换

### ImageDraw
`ImageDraw.ImageDraw`这是父类，下面的方法都是基于这个类，如果需要help用这个后面加方法或属性
PIL画盘，需要使用该函数将待处理图像放入Draw(Image)，返回一个句柄对象用来作画  
**常用方法介绍：**  
`ImageDraw.Draw()`画盘将图像包住  
`ImageDraw.textsize(text="content", font=fontObj)=>(width, height)`返回给定字符串的分辨率大小，以像素为单位  

`ImageDraw.text([loc], text="content", font=fontObj, fill=(color,color,color))`写文字  
`ImageFont.truetype(font=font_type, size=font_size)`设定字体,font传入字体路径    
`draw.rectangle([xy], outline=(color), fill=(color))`
功能：绘制一个长边形。
变量box是包含2元组[(x,y),…]或者数字[x,y,…]的任何序列对象。它应该包括2个坐标值(左上坐标axis=1和右下坐标axis=0),  
注意：当长方形没有没有被填充时，第二个坐标对定义了一个长方形外面的点。  
变量options可以绘制外框和矩形色块，**outline是指定外框的颜色**，fill是指定矩形色块填充颜色  
`draw.line([x1, y1, x2, y2], fill=color)`画线  
`Image.paste(img, loc)`将img粘贴到self上
* 粘贴新图片至图片中，loc 参数可以为 2-元组（upper, left）只需要传左上坐标就好了，或是 4-元组（left, upper, right, lower），或者是 None  
* 功能同上。不过是将指定位置填充为某种颜色。

In [3]:
def make_dirs(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
        
background_imgs_dir = "bg_img/"
fonts_dir = "fonts/"
handwriting_imgs_dir = "手写字体数据集/"
handwriting_set = os.listdir(handwriting_imgs_dir)
saved_path = "synthesis_imgs"
text_block_path = ""
make_dirs(saved_path) 


In [840]:
zhon_collection = "collect_chinese_word.txt"
with open(zhon_collections, 'r') as f:
    zhon_chars_collection = f.read(-1)
    
def select_text(zhon_chars_collection, mode='long', text_len=15, repeat=None):
    end_symbol = ['.','。','?']
    interval_symbol_collection = ['，','：','、','，', ',', '、','“']
    L1 = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't',
      'u', 'v', 'w', 'x', 'y', 'z',
      'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
      'U', 'V', 'W', 'X', 'Y', 'Z',
      '1', '2', '3', '4', '5', '6', '7', '8', '9', '0',
      '1.', '2.', '3.', '4.', '5.', '6.', '7.', '8.', '9.', '⒑', '⒒', '⒓', '⒔', '⒕', '⒖', '⒗', '⒘', '⒙', '⒚'                                                                                             
      "Ⅰ","Ⅱ", "Ⅲ", "Ⅳ", "Ⅴ", "Ⅵ", "Ⅶ", "Ⅷ", "Ⅸ", "Ⅹ", "Ⅺ", "Ⅻ",
      "ⅰ", "ⅱ", "ⅲ", "ⅳ", "ⅴ", "ⅵ", "ⅶ", "ⅷ", "ⅸ", "ⅹ",
      "⑴", "⑵", "⑶", "⑷", "⑸", "⑹", "⑺", "⑻", "⑼", "⑽", "⑾", "⑿", "⒀", "⒁", "⒂", "⒃", "⒄", "⒅", "⒆", "⒇",
      '一、', '二、', '三、', '四、', '五、', '六、', '七、', '八、', '九、', '十、',
      '㈠', '㈡', '㈢', '㈣', '㈤', '㈥', '㈦', '㈧', '㈨', '㈩']
    L2 = ["~", "～", "!", "@", "#", "$", "%", "^", "&", "*", "→", "↓", "：", ";","；","？", "/", "ㄨ", "°",
                  "＼", "\\", "、", "﹐","。",".",",",
                  '一', '二', '三', '四', '五', '六', '七', '八', '九', '十',
                  '1', '2', '3', '4', '5', '6', '7', '8', '9', '0',
                  "+", "-","=", ">", "─", "±", "^", "∩", "∈", "β", "÷", "≥", "α", "≤", "<", "‰", "П", "≠",
                  "￥", "℃", "☆", "◆", "■", "△", "◇", "√", "※", "∵", "匚", "阝", "刂", "パ", "é", "à", "ü", "ɡ", "ē", "ǒ","〉"]
    
    number_collection = []
    char_nums = len(zhon_chars_collection)
    block_repeat_freq = np.random.randint(1, 5) if repeat is None else repeat
    block_len = text_len //  block_repeat_freq
    text = []
    if mode == "long":
        start_char = np.random.randint(0, char_nums)
        text_cluster = ''
        for i in range(block_repeat_freq):
            interval_len = block_len + np.random.randint(-3, 3)
            text_block = zhon_chars_collection[start_char: min(char_nums, start_char + interval_len)]
            text += text_block 
            text += np.random.choice(interval_symbol_collection) if i < block_repeat_freq-1 else np.random.choice(end_symbol)
        text = np.random.choice(L1) + text if np.random.randint(0, 10) < 2 else text#一定概率添加头部文件
        return text
    elif mode == 'complex':
        text_cluster = ''
        for i in range(block_repeat_freq):
            start_char = np.random.randint(0, char_nums)
            interval_len = block_len + np.random.randint(-3, 3)
            text_cluster += zhon_chars_collection[start_char: min(char_nums, start_char + interval_len)] 
            text_cluster += np.random.choice(interval_symbol_collection) if i < block_repeat_freq-1 else np.random.choice(end_symbol)
            text.append(text_cluster)
        return text
    elif mode == 'single':
        for i in range(block_repeat_freq):
            start_char = np.random.randint(0, char_nums)
            text.append(zhon_chars_collection[start_char: min(char_nums, start_char + block_len)])
        return text
    else:
        return None
x = select_text(zhon_chars_collection, mode='single')
print(x)

['噩葵殃淤诠昵眸馁奄绽闱蛛矜馔遐']


In [ ]:
def motion_blur(image, degree=12, angle=45):
    image = np.array(image)

    # 这里生成任意角度的运动模糊kernel的矩阵， degree越大，模糊程度越高
    M = cv2.getRotationMatrix2D((degree / 2, degree / 2), angle, 1)
    motion_blur_kernel = np.diag(np.ones(degree))
    motion_blur_kernel = cv2.warpAffine(motion_blur_kernel, M, (degree, degree))

    motion_blur_kernel = motion_blur_kernel / degree
    blurred = cv2.filter2D(image, -1, motion_blur_kernel)

    # convert to uint8
    cv2.normalize(blurred, blurred, 0, 255, cv2.NORM_MINMAX)
    blurred = np.array(blurred, dtype=np.uint8)
    return blurred


def crop_img(img):
    """
    Return:(y1, x1, y2, x2)最大外接矩形的左上坐标和右下坐标
    """
    h, w = img.shape
    margin = np.random.randint(10, 20)#留出一点的空白部分,否则会导致识别出问题
    horiziontal_indicies = np.where(np.any(img, axis=0))[0]
    vertival_indicies = np.where(np.any(img, axis=1))[0]
    if horiziontal_indicies.shape[0]:
        x1, x2 = horiziontal_indicies[[0, -1]]
        y1, y2 = vertival_indicies[[0, -1]]
    else:
        x1, x2, y1, y2 = 0, 0, 0, 0
    x1 = max(x1 - margin, 0)#最大最小原则
    x2 = min(x2 + margin, w)
    return 255 - img[y1:y2, x1:x2] #直接返回crop好的图片

In [479]:
def resize_image(image, size=None):
    if isinstance(image, Image.Image):
        h, w = image.size
        scale_ratio = h / w
        w_after = round(size)
        h_after = round(w_after * scale_ratio)
        image.resize((w_after, h_after))
    elif isinstance(image, np.ndarray):
        h, w = image.shape
        print(h, w)
        scale_ratio = h / w
        w_after = round(size)
        h_after = round(w_after * scale_ratio)
        print(h_after, w_after)
        return cv2.resize(image, (w_after, h_after))#注意要返回对象才生效


150 250
60 100


(60, 100)

## TODO
* 使用PIL实现在图像上写一部分文字
** 读取背景图片
** 读取文字库txt，然后从中选取一部分区块文字，然后进行生成图片
** 选择文字区块，长度随机(1-40),字体大小要随机(字体大小要与背景图像尺寸相关)，字体类型随机，
* 读取手写字体并贴到打印字体的合成图片上(取最大外界矩形，判断一下文字的区间   )
* 打印体和手写体都需要确定location，并且形成标注文件


确定两头，字库不重复，保证每个字出现在开头结尾，判断每个字是否出现在头和尾

In [867]:
TL_set = []#读取所有的手写字体集
for dir in glob.glob("手写字体数据集/TL*"):
    for img in os.listdir(dir):
        TL_set.append(os.path.join(dir, img))


['手写字体数据集/TL01/b04010101_01.bmp', '手写字体数据集/TL01/b04010101_04.bmp', '手写字体数据集/TL01/b04010101_05.bmp', '手写字体数据集/TL01/b04010102_04.bmp', '手写字体数据集/TL01/b04010102_05.bmp', '手写字体数据集/TL01/b04010102_06.bmp', '手写字体数据集/TL01/b04010102_07.bmp', '手写字体数据集/TL01/b04010103_01.bmp', '手写字体数据集/TL01/b04010103_02.bmp', '手写字体数据集/TL01/b04010103_03.bmp', '手写字体数据集/TL01/b04010103_09.bmp', '手写字体数据集/TL01/b04010103_12.bmp', '手写字体数据集/TL01/b04010201_03.bmp', '手写字体数据集/TL01/b04010201_04.bmp', '手写字体数据集/TL01/b04010201_05.bmp', '手写字体数据集/TL01/b04010201_06.bmp', '手写字体数据集/TL01/b04010201_09.bmp', '手写字体数据集/TL01/b04010201_10.bmp', '手写字体数据集/TL01/b04010201_11.bmp', '手写字体数据集/TL01/b04010201_12.bmp', '手写字体数据集/TL01/b04010201_13.bmp', '手写字体数据集/TL01/b04010201_14.bmp', '手写字体数据集/TL01/b04010301_08.bmp', '手写字体数据集/TL01/b04010302_04.bmp', '手写字体数据集/TL01/b04010302_07.bmp', '手写字体数据集/TL01/b04010303_01.bmp', '手写字体数据集/TL01/b04010303_02.bmp', '手写字体数据集/TL01/b04010303_03.bmp', '手写字体数据集/TL01/b04010303_04.bmp', '手写字体数据集/TL01/b04010303_05.bmp', '手写字体数据集/

In [878]:
gen_nums = 1
margin_sapce = np.random.randint(15, 40)
text_mode = ['single', 'complex', 'long', 'space']
#---------
for i in range(gen_nums):
    saved_name = str(i) + '.bmp'
    back_img = Image.open(os.path.join(background_imgs_dir, np.random.choice(bg_imgs)))
    w_back, h_back = back_img.size
    line_nums = np.random.randint(5, 20)
    row_interval = np.random.randint()
    draw = ImageDraw.Draw(back_img)
    font_size = min(w_back // (2 * line_nums), h_back // (2 * line_nums))
    row_interval = font_size - np.random.randint(-5, 5)
    font_type = ImageFont.truetype(font=os.path.join(fonts_dir, random.choice(fonts)), size=font_size)
    x_loc, y_loc = (margin_sapce, margin_sapce)
    draw.textsize(text=)

['藕簋，', '藕簋，遇港，', '藕簋，遇港，坩垅拤坫垆抻?']
['彬', '俺', '忿', '螺', '“', '彬', '俺', '忿', '螺', '膀', '惫', '扔', '匪', '.']
['a年b定的一是不在有人c了中国大。']
['洳', '宬', '窀', '袆', '祢', '昶', '、', '洳', '宬', '窀', '袆', '祢', '“', '洳', '宬', '窀', '袆', '祢', '昶', '牁', '.']
['陟爲桧鸯恿晌骈喽淅澹叽桢刨忑忐']
['顽硅宛瘦剥睹、', '顽硅宛瘦剥睹、婺噫潞呱幡.']
['洼漕飓', '怛豉籁', '雳箸蚩', '嫡幢砸']
['乌', '赞', '锋', '返', '迫', ',', '乌', '赞', '锋', '返', '、', '乌', '赞', '锋', '：', '乌', '?']
['疑', '黑', '劳', '脑', '舰', '晚', '盘', '征', '波', '背', '访', '互', '败', '苦', '阶', '味', '。']
['蓿枵橪，', '蓿枵橪，匝桅椁绫桡、', '蓿枵橪，匝桅椁绫桡、腱^镬仡樾驽峒?']
['介博软欢害七良善移土课Z.']
['胄', '榛', '豢', '堑', '帔', '咙', '柚', '僭', '锵', '肮', '囿', '忤', '惴', '燮', '棹', '?']
['钤俅酊', '垓槿蛤', '入s加', '区管见']


In [ ]:
def draw_line_text(drawObj, font_size=20, font_obj=None, loc=[], texts=None):
    if texts =None:
        return loc[0], loc[1] + font_size
    else:
        for txt in texts:
            space = 20
            draw.text([loc[0], loc[1]+space], text=txt, font=font_obj, fill='black')
    for i 
    drawObj.text()
    
    
    for line in range(line_nums): 
    if x_loc > w_back or y_back > h_back:#边界检查
        break
    text_block = select_text(zhon_chars_collection, mode=np.random.choice(text_mode))
    for txt in text_block:
        if txt is None:

    print(text_block)

In [860]:
blank = Image.new("RGB", [1024, 768], "white")
drawObj = ImageDraw.Draw(blank)
drawObj.line([100, 100, 100, 600], fill='red')
drawObj.line([200,200,500,200], fill='black')
blank.show()

In [841]:
random.seed(666)

TL_set = glob.glob(handwriting_imgs_dir+'T*')#glob.glob()返回正则匹配到的结果，return一个结果list
blur_ratio = 0.1
text_block_path = "collect_chinese_word.txt"#文字路径
fonts = os.listdir(fonts_dir)
with open(text_block_path, "r") as f:
    items = f.readlines()
    
bg_imgs = os.listdir(background_imgs_dir)
sample = Image.open(os.path.join(background_imgs_dir, np.random.choice(bg_imgs)))

bg_h, bg_w = sample.size
new_one = Image.new('RGB', (bg_h, bg_w), 'white')
font_type = os.path.join(fonts_dir, random.choice(fonts))
scale_ratio = 1 / 40
base_size = int(bg_h * scale_ratio)
font_size = random.randint(0, 10) + base_size

blank = Image.new("RGB", [1024, 768], "white")
drawObj = ImageDraw.Draw(sample)
drawObj.line([100, 100, 100, 600], fill='red')
drawObj.line([200,200,200,500], fill='black')
text = "敢问路在何方"
font1 = ImageFont.truetype(font=font_type, size=font_size)#生成好了随机大小，类型的字体

bbs_list = []
for i in range(bg_h // font_size):
    random_interval = random.randint(0, font_size)
    if random_interval > bg_h:
        break
    h_loc = i * font_size
    w_txt, h_txt = drawObj.textsize(text=text, font=font1)

    drawObj.rectangle([0, h_loc, w_txt, h_loc + h_txt], outine=(255, 7, 4) )
    drawObj.text([0, h_loc], text, 'black', font=font1)
    if np.random.randint(0, 10) < 8:
        blank = Image.new('RGB', (w_txt, h_txt), 'red')
        sample.paste(blank, (w_txt, h_loc + h_txt))
Image.fromarray(sample_blurred).show()

NameError: name 'w_txt' is not defined

In [ ]:
def gen_txt_per_row():
    h_back, w_back = bg_img.size
    hw_ratio = h_back / w_back
    margin_ratio = 1/4
    y_start = margin_ratio * h_back + np.random.randint(0, 10)
    x_start = margin_ratio * hw_ratio * w_back + np.random.randint(0, 10)

In [ ]:
def gen_single_char_text():
    msTime = int(round(time.time()))

In [ ]:
def gen_text(img, first_end_text, text_block, start_char, ds):
    draw = ImageDraw.Draw(img)
    w_

In [ ]:
def motion_blur(image, degree=12, angle=45):
    image = np.array(image)

    # 这里生成任意角度的运动模糊kernel的矩阵， degree越大，模糊程度越高
    M = cv2.getRotationMatrix2D((degree / 2, degree / 2), angle, 1)
    motion_blur_kernel = np.diag(np.ones(degree))
    motion_blur_kernel = cv2.warpAffine(motion_blur_kernel, M, (degree, degree))

    motion_blur_kernel = motion_blur_kernel / degree
    blurred = cv2.filter2D(image, -1, motion_blur_kernel)

    # convert to uint8
    cv2.normalize(blurred, blurred, 0, 255, cv2.NORM_MINMAX)
    blurred = np.array(blurred, dtype=np.uint8)
    return blurred
sample.show()

### write测试
'r'：读  

'w'：写  

'a'：追加  

'r+' == r+w（可读可写，文件若不存在就报错(IOError)）  

'w+' == w+r（可读可写，文件若不存在就创建） 

'a+' ==a+r（可追加可写，文件若不存在就创建） 

In [465]:
with open("text_write.txt", "a+") as f:
    print(f.name)
    for i in range(20):
        s = "s" * i
        f.write(s + '\n')
with open("text_write.txt", 'r+') as f:
    content = f.read(-1)
print(content)
    

text_write.txt

s
ss
sss
ssss
sssss
ssssss
sssssss
ssssssss
sssssssss
ssssssssss
sssssssssss
ssssssssssss
sssssssssssss
ssssssssssssss
sssssssssssssss
ssssssssssssssss
sssssssssssssssss
ssssssssssssssssss
sssssssssssssssssss

s
ss
sss
ssss
sssss
ssssss
sssssss
ssssssss
sssssssss
ssssssssss
sssssssssss
ssssssssssss
sssssssssssss
ssssssssssssss
sssssssssssssss
ssssssssssssssss
sssssssssssssssss
ssssssssssssssssss
sssssssssssssssssss



In [ ]:

start = 0
newCanvas = Image.new("RGB", (2512,2512), 'white')#新建背景画布

# newCanvas.show()
h_back, w_back = newCanvas.size
font_type = os.path.join("fonts", np.random.choice(os.listdir("fonts/")))
font_size = np.random.randint(15, 30)
margin_ratio = 1 / 8
texts = zhon_chars[start:start + word_len]

# ImageFont = ImageFont.truetype(font=font, size=np.random.randint(5,10)) 
font = ImageFont.truetype(font=font_type, size=font_size)#生成好了随机大小，类型的字体
x_start, y_start = int(margin_ratio * w_back), int(margin_ratio * h_back)
draw = ImageDraw.Draw(newCanvas)
w_txt, h_txt = draw.textsize(font=font, text=texts)
with Image.open("hand_sample.bmp") as hand_img:
    w_hand, h_hand = hand_img.size
    hw_ratio = h_hand / w_hand
    draw.text(xy=(x_start, y_start), text=texts, font=font, fill=(255,0,0))
    hand_img.resize((int(hw_ratio * h_back * 1/5), int(w_back * 1/5)))
    newCanvas.paste(hand_img, (x_start + w_txt, y_start + h_txt) )
    newCanvas.show()